# Import

In [1]:
import pandas as pd
import difflib

# Data Preparation

In [2]:
left_data = pd.read_csv("C:/Users/WHR/Desktop/final_project/group-project-spring-2024/Data/GroupProject/entity-resolution/left_dataset.csv")
right_data = pd.read_csv("C:/Users/WHR/Desktop/final_project/group-project-spring-2024/Data/GroupProject/entity-resolution/right_dataset.csv")

In [3]:
left = pd.DataFrame(left_data)
right = pd.DataFrame(right_data)

In [4]:
print("left", len(left))
print("right", len(right))

left 94585
right 91791


In [5]:
left['postal_code'] = left['postal_code'].astype(str)
right['zip_code'] = right['zip_code'].astype(str)

In [6]:
left['postal_code'] = left['postal_code'].str[:5]
right['zip_code'] = right['zip_code'].str[:5]

In [7]:
left['combined_address'] = left['address'] + ' ' + left['city'] + ' ' + left['state'] + ' ' + left['postal_code']
right['combined_address'] = right['address'] + ' ' + right['city'] + ' ' + right['state'] + ' ' + right['zip_code']

In [8]:
for col in left.columns:
    left[col] = left[col].apply(lambda x: x.lower() if type(x) == str else x)
for col in right.columns:
    right[col] = right[col].apply(lambda x: x.lower() if type(x) == str else x)

In [9]:
cities_left_count = left['city'].nunique()
cities_right_count = right['city'].nunique()
print(cities_left_count, cities_right_count)

794 44


In [10]:
cities_left = left['city'].unique()
left_list = cities_left.tolist()

cities_right = right['city'].unique()
right_list = cities_right.tolist()

# Processing

In [11]:
def find_matches(df1, df2, key1_primary, key2_primary, key1_secondary, key2_secondary, threshold=0.8):
    matches = []
    for i, row1 in df1.iterrows():
        best_score = threshold
        best_match = None
        for j, row2 in df2.iterrows():
            primary1 = str(row1[key1_primary]) if pd.notna(row1[key1_primary]) else ''
            primary2 = str(row2[key2_primary]) if pd.notna(row2[key2_primary]) else ''
            score = difflib.SequenceMatcher(None, primary1, primary2).ratio()
            if score > best_score:
                best_score = score
                best_match = (row1['entity_id'], row2['business_id'], score)
        if best_match:
            matches.append(best_match)
        else:
            secondary1 = str(row1[key1_secondary]) if pd.notna(row1[key1_secondary]) else ''
            secondary2 = str(row2[key2_secondary]) if pd.notna(row2[key2_secondary]) else ''
            score = difflib.SequenceMatcher(None, secondary1, secondary2).ratio()
            if score > best_score:
                best_score = score
                best_match = (row1['entity_id'], row2['business_id'], score)
            if best_match:
                matches.append(best_match)
    return matches

In [12]:
def match_all(df1, df2, city_list):
    
    all_matches = []
    
    for city in city_list:
        
        left_data = df1[df1['city'] == city]
        right_data = df2[df2['city'] == city]
        
        refined_matches = find_matches(left_data, right_data, 'name', 'name', 'combined_address', 'combined_address')
        all_matches.append(refined_matches)
        
    return all_matches

In [13]:
result = match_all(left, right, right_list)

In [14]:
len(result)

44

In [15]:
count = 0
for match in result:
    count += len(match)
print(count)

7985


In [16]:
result

[[(101, 26632, 0.8607594936708861),
  (731, 26632, 0.9425287356321839),
  (876, 1591, 0.972972972972973),
  (1389, 9173, 0.8181818181818182),
  (1589, 17549, 0.8333333333333334),
  (1899, 26632, 0.8607594936708861),
  (2475, 3354, 0.9310344827586207),
  (3014, 7565, 0.8108108108108109),
  (3701, 15112, 0.8253968253968254),
  (3727, 20182, 0.8461538461538461),
  (4287, 1904, 0.9130434782608695),
  (4362, 7167, 0.8636363636363636),
  (4513, 192, 0.875),
  (4860, 6730, 0.9032258064516129),
  (4995, 7565, 0.9459459459459459),
  (5107, 26632, 0.8607594936708861),
  (5479, 20837, 0.8571428571428571),
  (5942, 2677, 0.9180327868852459),
  (6145, 19659, 0.9166666666666666),
  (6169, 23611, 0.8666666666666667),
  (6207, 1391, 0.8148148148148148),
  (6808, 24609, 1.0),
  (6927, 945, 0.8852459016393442),
  (7872, 26632, 0.8607594936708861),
  (8031, 26632, 0.896551724137931),
  (8329, 26632, 0.8607594936708861),
  (8378, 3099, 0.9375),
  (8873, 26632, 0.8607594936708861),
  (8944, 13297, 0.818181